In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import shutil
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.image import resize, convert_image_dtype, decode_jpeg
from tensorflow.io import read_file
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, UpSampling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2

In [ ]:
clear_dir = '/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Clear/Foreign'
distorted_dir = '/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Distorted/Distorted'

In [ ]:
clear_paths = sorted([os.path.join(clear_dir, name) for name in os.listdir(clear_dir)])

In [ ]:
cl_p = []
dis_p = []

In [ ]:
for clear_path in clear_paths[:50]:
    clear_name = os.path.basename(clear_path)[:-4]
    for i in range(50):
        dis_name = f"{clear_name}_{i}.jpg"
        dis_path = os.path.join(distorted_dir, dis_name)
        cl_p.append(clear_path)
        dis_p.append(dis_path)

In [ ]:
dataset = Dataset.from_tensor_slices((cl_p, dis_p))

In [ ]:
img_size = (256, 256)
batch_size = 8

dataset = dataset.map(lambda cl_p, dis_p: (
    resize(convert_image_dtype(decode_jpeg(read_file(dis_p), channels=3), tf.float32), img_size),
    resize(convert_image_dtype(decode_jpeg(read_file(cl_p), channels=3), tf.float32), img_size)
), num_parallel_calls=tf.data.AUTOTUNE)

# Denoising AE

In [ ]:
inp = Input(shape=(256, 256, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inp)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
out = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

dae = Model(inp, out)

In [ ]:
dae.compile(optimizer='adam', loss='mse')

In [ ]:
train_size = int(0.8 * len(cl_p))
val_size = int(0.1 * len(cl_p))

In [ ]:
train_ds = dataset.take(train_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = dataset.skip(train_size).take(val_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = dataset.skip(train_size + val_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
num_images = sum(1 for _ in train_ds)
print(f"Number of images in train_ds: {num_images}")

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
for distorted_batch, clear_batch in train_ds.take(1):
    print(f"Distorted batch shape: {distorted_batch.shape}")
    print(f"Clear batch shape: {clear_batch.shape}")

In [ ]:
dae.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[early_stopping])

In [ ]:
dae.save('/kaggle/working/dae.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for distorted_batch, clear_batch in test_ds.take(1):
    denoised_batch = dae.predict(distorted_batch)
    num_images = 5
    plt.figure(figsize=(15, 5))

    for i in range(num_images):

        plt.subplot(3, num_images, i + 1)
        plt.imshow(clear_batch[i])
        plt.title("Clear Image")
        plt.axis("off")

        plt.subplot(3, num_images, i + 1 + num_images)
        plt.imshow(distorted_batch[i])
        plt.title("Distorted Image")
        plt.axis("off")

        plt.subplot(3, num_images, i + 1 + 2 * num_images)
        plt.imshow(denoised_batch[i])
        plt.title("Denoised Image")
        plt.axis("off")

    plt.tight_layout()
    plt.show()
    break 

# UNet

In [ ]:
inp = Input(shape=(256,256,3))

c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inp)
c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)

c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)

c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
p4 = MaxPooling2D((2, 2))(c4)

bottleneck = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
bottleneck = Conv2D(1024, (3, 3), activation='relu', padding='same')(bottleneck)

u1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bottleneck)
u1 = concatenate([u1, c4])
c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(u1)
c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

u2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u2 = concatenate([u2, c3])
c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(u2)
c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u3 = concatenate([u3, c2])
c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

u4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u4 = concatenate([u4, c1])
c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

out = Conv2D(3, (1, 1), activation='sigmoid')(c8)

In [ ]:
unet = Model(inp, out)

In [ ]:
unet.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
unet.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[early_stopping])

In [ ]:
for distorted_images, clear_images in test_ds.take(1):
    denoised_images = unet.predict(distorted_images)

    # Plot results
    plt.figure(figsize=(10, 5))
    for i in range(3):
        plt.subplot(3, 3, i * 3 + 1)
        plt.imshow(distorted_images[i])
        plt.title("Distorted")
        plt.axis('off')

        plt.subplot(3, 3, i * 3 + 2)
        plt.imshow(denoised_images[i])
        plt.title("Denoised")
        plt.axis('off')

        plt.subplot(3, 3, i * 3 + 3)
        plt.imshow(clear_images[i])
        plt.title("Clear")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
unet.save('/kaggle/working/unet.h5')

# SCUNet

In [ ]:
op_dir = '/kaggle/working' 

clear_op_dir = os.path.join(op_dir, 'clear_images')
distorted_op_dir = os.path.join(op_dir, 'distorted_images')

os.makedirs(clear_op_dir, exist_ok=True)
os.makedirs(distorted_op_dir, exist_ok=True)

for i in range(1, 51):
    clear_img_name = f'F ({i}).jpg'
    clear_img_path = os.path.join(clear_dir, clear_img_name)

    if os.path.exists(clear_img_path):
        shutil.copy(clear_img_path, os.path.join(clear_op_dir, clear_img_name))

    for j in range(50):
        distorted_img_name = f'F ({i})_{j}.jpg'
        distorted_img_path = os.path.join(distorted_dir, distorted_img_name)

        if os.path.exists(distorted_img_path):
            shutil.copy(distorted_img_path, os.path.join(distorted_op_dir, distorted_img_name))

In [ ]:
!git clone https://github.com/cszn/SCUNet.git

In [ ]:
%cd SCUNet

In [ ]:
!python main_download_pretrained_models.py --models "SCUNet" --model_dir "model_zoo"

### Gaussian Noise

In [ ]:
%pip install thop einops

In [ ]:
!python main_test_scunet_color_gaussian.py --model_name scunet_color_25 --noise_level_img 25 --testset_name /kaggle/working/distorted_images

In [ ]:
clear_images_dir = '/kaggle/working/clear_images'
denoised_images_dir = '/kaggle/working/distorted_images_scunet_color_25'

clear_images = sorted(os.listdir(clear_images_dir))
denoised_images = sorted(os.listdir(denoised_images_dir))

def display_images(clear, denoised):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(clear, cv2.COLOR_BGR2RGB))
    plt.title('Clear Image')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB))
    plt.title('Denoised Image')
    plt.axis('off')
    
    plt.show()

num_images = min(10, len(clear_images))

for i in range(num_images):
    clear_image = cv2.imread(os.path.join(clear_images_dir, clear_images[i]))
    
    for j in range(50):
        distorted_image_index = i * 50 + j
        if distorted_image_index < len(denoised_images):
            denoised_image = cv2.imread(os.path.join(denoised_images_dir, denoised_images[distorted_image_index]))

            if clear_image is None or denoised_image is None:
                continue
            
            display_images(clear_image, denoised_image)
            break

### Blind (PSNR)

In [ ]:
!python main_test_scunet_real_application.py --model_name scunet_color_real_psnr --testset_name /kaggle/working/distorted_images

In [ ]:
clear_images_dir = '/kaggle/working/clear_images'
denoised_images_dir = '/kaggle/working/distorted_images_scunet_c_25'

clear_images = sorted(os.listdir(clear_images_dir))
denoised_images = sorted(os.listdir(denoised_images_dir))

def display_images(clear, denoised):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(clear, cv2.COLOR_BGR2RGB))
    plt.title('Clear Image')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB))
    plt.title('Denoised Image')
    plt.axis('off')
    
    plt.show()

num_images = min(10, len(clear_images))

for i in range(num_images):
    clear_image = cv2.imread(os.path.join(clear_images_dir, clear_images[i]))
    
    for j in range(50):
        distorted_image_index = i * 50 + j
        if distorted_image_index < len(denoised_images):
            denoised_image = cv2.imread(os.path.join(denoised_images_dir, denoised_images[distorted_image_index]))

            if clear_image is None or denoised_image is None:
                continue
            
            display_images(clear_image, denoised_image)
            break

### Blind (GAN)

In [ ]:
!python main_test_scunet_real_application.py --model_name scunet_color_real_gan --testset_name /kaggle/working/distorted_images

In [ ]:
clear_images_dir = '/kaggle/working/clear_images'
denoised_images_dir = '/kaggle/working/distorted_images_scunet_c_25'

clear_images = sorted(os.listdir(clear_images_dir))
denoised_images = sorted(os.listdir(denoised_images_dir))

def display_images(clear, denoised):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(clear, cv2.COLOR_BGR2RGB))
    plt.title('Clear Image')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB))
    plt.title('Denoised Image')
    plt.axis('off')
    
    plt.show()

num_images = min(10, len(clear_images))

for i in range(num_images):
    clear_image = cv2.imread(os.path.join(clear_images_dir, clear_images[i]))
    
    for j in range(50):
        distorted_image_index = i * 50 + j
        if distorted_image_index < len(denoised_images):
            denoised_image = cv2.imread(os.path.join(denoised_images_dir, denoised_images[distorted_image_index]))

            if clear_image is None or denoised_image is None:
                continue
            
            display_images(clear_image, denoised_image)
            break

# Bayesian Reconstruction using Generative Model (BRGM)

In [ ]:
!conda config --add channels defaults

In [ ]:
!conda create -y -n brgm python=3.7
!source activate brgm

In [ ]:
!conda install -y -n brgm pytorch==1.7.1 -c pytorch 

In [ ]:
%cd /kaggle/working/brgm

# DnCNN

In [ ]:
def build_dncnn(depth=17, filters=64, image_channels=3, kernel=3):
    inp = Input(shape=(256, 256, image_channels), name='input')

    x = Conv2D(filters, (kernel, kernel), padding='same', activation='relu')(inp)

    for _ in range(depth - 2):
        x = Conv2D(filters, (kernel, kernel), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = Conv2D(image_channels, (kernel, kernel), padding='same')(x)

    out = Subtract(name='residual')([inp, x])
    
    model = Model(inputs=inp, outputs=out, name='DnCNN')
    return model

In [ ]:
dncnn = build_dncnn(depth=17, filters=64, image_channels=3, kernel=3)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
dncnn.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

In [ ]:
dncnn.fit(train_ds, validation_data=val_ds, epochs=50, verbose=1, callbacks=[early_stopping])

In [ ]:
for distorted_images, clear_images in test_ds.take(1):
    denoised_images = dncnn.predict(distorted_images)

    # Plot results
    plt.figure(figsize=(10, 5))
    for i in range(3):
        plt.subplot(3, 3, i * 3 + 1)
        plt.imshow(distorted_images[i])
        plt.title("Distorted")
        plt.axis('off')

        plt.subplot(3, 3, i * 3 + 2)
        plt.imshow(denoised_images[i])
        plt.title("Denoised")
        plt.axis('off')

        plt.subplot(3, 3, i * 3 + 3)
        plt.imshow(clear_images[i])
        plt.title("Clear")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
dncnn.save('/kaggle/working/dncnn.h5')

# Restormer

In [ ]:
if os.path.isdir('Restormer'):
  !rm -r Restormer

!git clone https://github.com/swz30/Restormer.git
%cd Restormer

In [ ]:
task = 'Real_Denoising'

if task == 'Real_Denoising':
  !wget https://github.com/swz30/Restormer/releases/download/v1.0/real_denoising.pth -P Denoising/pretrained_models

In [ ]:
!pip install pytest-shutil

In [ ]:
import shutil

In [ ]:
!rm -r demo/*
!wget https://github.com/swz30/Restormer/releases/download/v1.0/sample_images.zip -P demo

In [ ]:
!pip install natsort

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np

def get_weights_and_parameters(task, parameters):
    if task == 'Motion_Deblurring':
        weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
    elif task == 'Single_Image_Defocus_Deblurring':
        weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
    elif task == 'Deraining':
        weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
    elif task == 'Real_Denoising':
        weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
        parameters['LayerNorm_type'] =  'BiasFree'
    return weights, parameters

parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
model = load_arch['Restormer'](**parameters)
model.cuda()

checkpoint = torch.load(weights)
model.load_state_dict(checkpoint['params'])
model.eval()

In [ ]:
input_dir = 'demo/sample_images/'+task+'/degraded'
out_dir = 'demo/sample_images/'+task+'/restored'
os.makedirs(out_dir, exist_ok=True)
extensions = ['jpg', 'JPG', 'png', 'PNG', 'jpeg', 'JPEG', 'bmp', 'BMP']
files = natsorted(glob(os.path.join(input_dir, '*')))

img_multiple_of = 8

print(f"\n ==> Running {task} with weights {weights}\n ")
with torch.no_grad():
  for filepath in tqdm(files):
      # print(file_)
      torch.cuda.ipc_collect()
      torch.cuda.empty_cache()
      img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
      input_ = torch.from_numpy(img).float().div(255.).permute(2,0,1).unsqueeze(0).cuda()

      # Pad the input if not_multiple_of 8
      h,w = input_.shape[2], input_.shape[3]
      H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
      padh = H-h if h%img_multiple_of!=0 else 0
      padw = W-w if w%img_multiple_of!=0 else 0
      input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

      restored = model(input_)
      restored = torch.clamp(restored, 0, 1)

      # Unpad the output
      restored = restored[:,:,:h,:w]

      restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
      restored = img_as_ubyte(restored[0])

      filename = os.path.split(filepath)[-1]
      cv2.imwrite(os.path.join(out_dir, filename),cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))

In [ ]:
op_dir = '/kaggle/working'
clear_dir = '/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Clear/Foreign'
distorted_dir ='/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Distorted/Distorted'

clear_op_dir = os.path.join(op_dir, 'clear_images')
distorted_op_dir = os.path.join(op_dir, 'distorted_images')

os.makedirs(clear_op_dir, exist_ok=True)
os.makedirs(distorted_op_dir, exist_ok=True)

for i in range(1, 51):
    clear_img_name = f'F ({i}).jpg'
    clear_img_path = os.path.join(clear_dir, clear_img_name)

    if os.path.exists(clear_img_path):
        shutil.copy(clear_img_path, os.path.join(clear_op_dir, clear_img_name))

    for j in range(50):
        distorted_img_name = f'F ({i})_{j}.jpg'
        distorted_img_path = os.path.join(distorted_dir, distorted_img_name)

        if os.path.exists(distorted_img_path):
            shutil.copy(distorted_img_path, os.path.join(distorted_op_dir, distorted_img_name))

In [ ]:
input_dir = '/kaggle/working/distorted_images'
clear_op_dir = '/kaggle/working/clear_images'
restormer_op_dir = '/kaggle/working/outputreso'
out_dir = '/kaggle/working/output_images'

# Create necessary directories with exist_ok=True
os.makedirs(clear_op_dir, exist_ok=True)
os.makedirs(restormer_op_dir, exist_ok=True)  # No assignment here, just creating the directory
os.makedirs(out_dir, exist_ok=True)

extensions = ['jpg', 'JPG', 'png', 'PNG', 'jpeg', 'JPEG', 'bmp', 'BMP']
files = natsorted(glob(os.path.join(input_dir, '*')))

img_multiple_of = 8

print(f"\n ==> Running {task} with weights {weights}\n ")
with torch.no_grad():
    for filepath in tqdm(files):
        torch.cuda.ipc_collect()
        torch.cuda.empty_cache()
        
        img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
        input_ = torch.from_numpy(img).float().div(255.).permute(2, 0, 1).unsqueeze(0).cuda()

        # Pad the input if not multiple of 8
        h, w = input_.shape[2], input_.shape[3]
        H, W = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of, ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
        padh = H - h if h % img_multiple_of != 0 else 0
        padw = W - w if w % img_multiple_of != 0 else 0
        input_ = F.pad(input_, (0, padw, 0, padh), 'reflect')

        restored = model(input_)
        restored = torch.clamp(restored, 0, 1)

        # Unpad the output
        restored = restored[:, :, :h, :w]

        restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
        restored = img_as_ubyte(restored[0])

        filename = os.path.split(filepath)[-1]
        cv2.imwrite(os.path.join(restormer_op_dir, filename), cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))
